In [ ]:
import cv2
import numpy as np
import os
import json
import matplotlib.pyplot as plt
from PIL import Image

In [ ]:
IMAGES_PATH = '/media/pupa/DataStorage/datasets/'
ANNOTATIONS_PATH = './annotations/'

def draw_landmarks_crop(img, landmarks):
    for landmark in landmarks:
        x, y = landmark
        x, y = int(x), int(y)
        img = cv2.circle(img, (x, y), 3, (0, 0, 255), -1)
    return img

def draw_bboxes(img, bbox):
    left, top, right, bottom = bbox['left'], bbox['top'], bbox['right'], bbox['bottom']
    img = cv2.rectangle(img, (left, top), (right, bottom), (255, 0, 0), 1)
    return img

def read_sample(sample_name):
    img_path = os.path.join(IMAGES_PATH, sample_name + '.jpg')

    if sample_name.startswith('WFLW/'):
        img_path = img_path.replace('WFLW/', 'WFLW/WFLW_images/')
    elif sample_name.startswith('ffhq'):
        img_path = img_path.replace('ffhq', 'ffhq-dataset/images1024x1024')

    if not os.path.exists(img_path):
        img_path = img_path[:-4] + '.png'
        assert os.path.exists(img_path), 'Image not found ' + img_path
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    annotation_path = os.path.join(ANNOTATIONS_PATH, sample_name + '.json')
    assert os.path.exists(annotation_path), 'Landmarks not found'
    with open(annotation_path, 'r') as f:
        annotations = json.load(f)

    return img, annotations

def convert_to_image_coords(img, bbox, landmarks):
    width, height = img.shape[1], img.shape[0]
    left, top, right, bottom = bbox['left'], bbox['top'], bbox['right'], bbox['bottom']
    left, top, right, bottom = int(left * width), int(top * height), int(right * width), int(bottom * height)

    box_width = right - left
    box_height = bottom - top

    landmarks = [(int(l[0] * box_width + left), int(l[1] * box_height + top)) for l in landmarks]
    bbox = {'left': left, 'top': top, 'right': right, 'bottom': bottom}
    return bbox, landmarks


sample_img, sample_an = read_sample('300W/afw/170817766_1')
# sample_img, sample_an = read_sample('WFLW/61--Street_Battle/61_Street_Battle_streetfight_61_10')
# sample_img, sample_an = read_sample('celeba_hq/train/female/000085')
# sample_img, sample_an = read_sample('ffhq/00000/00000')
img = sample_img.copy()
for face in sample_an:
    bbox, landmarks = convert_to_image_coords(img, face['bbox'], face['landmarks'])
    img = draw_bboxes(img, bbox)
    img = draw_landmarks_crop(img, landmarks)

plt.imshow(img)